In [ ]:
import org.apache.spark.sql._

val spark = SparkSession.builder()
    .master("local[*]")
    .appName("hw6")
    .getOrCreate()

import spark.implicits._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.lower
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.regexp_replace

In [ ]:
var file = spark.read.options(Map("inferSchema"->"true","delimiter"->",","header"->"true"))
  .csv("tripadvisor_hotel_reviews.csv")

In [ ]:
file = file.drop("Rating")
file = file.withColumn("Review", regexp_replace(file("Review"), "[,.!?:;*-_]", ""))
file = file.withColumn("Review", regexp_replace(file("Review"), "[0-9]", ""))
file = file.withColumn("Review", lower($"Review"))

In [ ]:
file = file.withColumn("Review_list", split(col("Review"), pattern=" ").as[Array[String]])
file = file.withColumn("len_review_list", size(col("Review_list")))
file = file.withColumn("review_id", monotonically_increasing_id())
file.show(3)

In [ ]:
var tf = file.select($"review_id", $"len_review_list", explode($"Review_list"))
tf = tf.withColumnRenamed("col","word_tf")
tf = tf.groupBy("review_id", "len_review_list", "word_tf").count().orderBy("review_id")
tf = tf.withColumn("tf", col("count")/col("len_review_list"))
tf.show(6)

In [ ]:
var idf = tf.groupBy("word_tf").count()
idf = idf.withColumnRenamed("word_tf","word_idf")
idf.show(6)

In [ ]:
val len_file = file.count

In [ ]:
idf = idf.withColumn("idf", log(lit(len_file))-log(col("count")))
idf.show(6)

In [ ]:
var top_100 = idf.orderBy(desc("count")).filter($"word_idf" !== "").limit(100).drop("count")
top_100.show(5)

In [ ]:
var tf_top_100 = top_100.join(tf, top_100("word_idf") === tf("word_tf"))
tf_top_100 = tf_top_100.drop("word_idf", "idf")
tf_top_100.show(10)

In [ ]:
var tf_idf = tf_top_100.join(idf, tf_top_100("word_tf")===idf("word_idf"))
tf_idf = tf_idf.withColumn("tf-idf", col("tf")*col("idf"))
tf_idf = tf_idf.drop("len_review_list", "word_tf", "count", "tf", "idf")
tf_idf = tf_idf.withColumnRenamed("word_idf","word")
tf_idf.show(3)

In [ ]:
val pivot_table = tf_idf.groupBy("review_id").pivot("word").sum("tf-idf")

In [ ]:
pivot_table.show(1)